# Bitcoin Abnormal Activity Tracker

This notebook demonstrates how to use the Bitcoin Anomaly Bot to detect unusual activity including:
- **Price anomalies** - sudden spikes or drops using Z-score analysis
- **Volume anomalies** - unusual trading volume surges
- **Volatility spikes** - abnormal price swings
- **Whale activity** - large transactions on the blockchain
- **RSI extremes** - overbought/oversold conditions
- **Bollinger Band breakouts** - price breaking technical levels

In [ ]:
# Import the bot and dependencies
import sys
sys.path.insert(0, '.')

from strategies.bitcoin_anomaly_bot import (
    BitcoinAnomalyBot,
    BotConfig,
    MultiTimeframeAnalysis,
    AnomalyType,
    AlertSeverity
)
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import json

## 1. Configure the Bot

Adjust thresholds based on your sensitivity preferences:

In [ ]:
# Create custom configuration
config = BotConfig(
    symbol="BTC-USD",
    lookback_days=90,
    update_interval_seconds=300,  # 5 minutes
    
    # Anomaly detection thresholds
    price_zscore_threshold=2.5,      # Standard deviations for price
    volume_zscore_threshold=3.0,     # Standard deviations for volume
    volatility_zscore_threshold=2.5, # Standard deviations for volatility
    
    price_change_threshold_24h_pct=5.0,  # 5% price change triggers alert
    volume_spike_multiplier_24h=3.0,     # 3x average volume
    
    # Technical indicators
    bollinger_period=20,
    bollinger_std=2.0,
    rsi_period=14,
    rsi_oversold=30.0,
    rsi_overbought=70.0,
    
    # Multi-timeframe settings
    rsi_periods=[6, 14, 24],
    ma_periods=[20, 50, 100, 200],
    fng_history_days=30,
    
    # Whale tracking
    whale_threshold_btc=100.0,  # Transactions > 100 BTC
    
    # Alert settings
    enable_console=True,
    enable_email=False,
    enable_discord=False,
)

# Create bot instance
bot = BitcoinAnomalyBot(config)
print("Bot initialized successfully!")

## 2. Fetch Market Data

In [ ]:
# Fetch Bitcoin price data
df = bot.data_fetcher.get_price_data(period="30d", interval="1h")
print(f"Fetched {len(df)} data points")
df.tail(10)

In [ ]:
# Get current market summary
summary = bot.get_market_summary()
print(json.dumps(summary, indent=2, default=str))

## 3. Visualize Price Data with Anomaly Indicators

In [ ]:
def create_anomaly_chart(df: pd.DataFrame, config: BotConfig) -> go.Figure:
    """Create an interactive chart showing price, volume, and anomaly zones."""
    
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.4, 0.2, 0.2, 0.2],
        subplot_titles=('BTC Price with Bollinger Bands', 'Volume', 'RSI', 'Volatility')
    )
    
    # Calculate indicators
    prices = df['close']
    
    # Bollinger Bands
    bb_period = config.bollinger_period
    bb_std = config.bollinger_std
    df['bb_middle'] = prices.rolling(window=bb_period).mean()
    df['bb_std'] = prices.rolling(window=bb_period).std()
    df['bb_upper'] = df['bb_middle'] + (bb_std * df['bb_std'])
    df['bb_lower'] = df['bb_middle'] - (bb_std * df['bb_std'])
    
    # RSI
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=config.rsi_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=config.rsi_period).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # Volatility
    df['returns'] = prices.pct_change()
    df['volatility'] = df['returns'].rolling(window=20).std() * 100
    
    # Z-scores for anomaly detection
    df['price_zscore'] = (prices - prices.rolling(50).mean()) / prices.rolling(50).std()
    df['volume_zscore'] = (df['volume'] - df['volume'].rolling(50).mean()) / df['volume'].rolling(50).std()
    
    x_axis = df['datetime'] if 'datetime' in df.columns else df.index
    
    # Row 1: Price with Bollinger Bands
    fig.add_trace(
        go.Scatter(x=x_axis, y=df['bb_upper'], name='Upper BB',
                   line=dict(color='rgba(128,128,128,0.3)'), showlegend=False),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=x_axis, y=df['bb_lower'], name='Lower BB',
                   line=dict(color='rgba(128,128,128,0.3)'), fill='tonexty',
                   fillcolor='rgba(128,128,128,0.1)', showlegend=False),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=x_axis, y=prices, name='BTC Price',
                   line=dict(color='#1f77b4', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=x_axis, y=df['bb_middle'], name='BB Middle',
                   line=dict(color='orange', dash='dash')),
        row=1, col=1
    )
    
    # Highlight anomalies on price chart
    anomaly_mask = abs(df['price_zscore']) > config.price_zscore_threshold
    fig.add_trace(
        go.Scatter(x=x_axis[anomaly_mask], y=prices[anomaly_mask],
                   mode='markers', name='Price Anomaly',
                   marker=dict(size=10, color='red', symbol='circle')),
        row=1, col=1
    )
    
    # Row 2: Volume
    colors = ['red' if z > config.volume_zscore_threshold else 'blue' 
              for z in df['volume_zscore'].fillna(0)]
    fig.add_trace(
        go.Bar(x=x_axis, y=df['volume'], name='Volume',
               marker_color=colors, showlegend=False),
        row=2, col=1
    )
    
    # Row 3: RSI
    fig.add_trace(
        go.Scatter(x=x_axis, y=df['rsi'], name='RSI',
                   line=dict(color='purple')),
        row=3, col=1
    )
    fig.add_hline(y=config.rsi_overbought, line_dash="dash", line_color="red",
                  annotation_text="Overbought", row=3, col=1)
    fig.add_hline(y=config.rsi_oversold, line_dash="dash", line_color="green",
                  annotation_text="Oversold", row=3, col=1)
    
    # Row 4: Volatility
    fig.add_trace(
        go.Scatter(x=x_axis, y=df['volatility'], name='Volatility (%)',
                   line=dict(color='orange'), fill='tozeroy'),
        row=4, col=1
    )
    
    fig.update_layout(
        title='Bitcoin Anomaly Detection Dashboard',
        height=900,
        showlegend=True,
        template='plotly_white'
    )
    
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="RSI", row=3, col=1)
    fig.update_yaxes(title_text="Volatility (%)", row=4, col=1)
    
    return fig

# Create and display chart
fig = create_anomaly_chart(df.copy(), config)
fig.show()

## 4. Run Anomaly Detection (Single Check)

In [ ]:
# Run a single anomaly check
anomalies = bot.check_once()

# Display detected anomalies
if anomalies:
    print(f"\nDetected {len(anomalies)} anomalies:\n")
    for i, anomaly in enumerate(anomalies, 1):
        print(f"{i}. [{anomaly.severity.value.upper()}] {anomaly.anomaly_type.value}")
        print(f"   {anomaly.description}")
        print(f"   Current: {anomaly.current_value:,.2f} | Threshold: {anomaly.threshold_value:,.2f}")
        if anomaly.z_score:
            print(f"   Z-Score: {anomaly.z_score:.2f}")
        print()
else:
    print("No anomalies detected - market is behaving normally.")

## 5. Check for Whale Activity

In [ ]:
# Fetch whale transactions from blockchain
whale_txs = bot.data_fetcher.get_whale_transactions()

if whale_txs:
    print(f"Found {len(whale_txs)} whale transactions (>100 BTC):\n")
    for tx in whale_txs[:10]:  # Show top 10
        print(f"Hash: {tx['hash']}")
        print(f"Amount: {tx['amount_btc']:.2f} BTC")
        print(f"Time: {tx['time']}")
        print(f"Inputs: {tx['inputs']} | Outputs: {tx['outputs']}")
        print("-" * 50)
else:
    print("No whale transactions detected in recent blocks.")

## 6. Fear & Greed Index

In [ ]:
# Get crypto Fear & Greed Index
fng = bot.data_fetcher.get_fear_greed_index()

if fng:
    value = fng['value']
    classification = fng['classification']
    
    # Color based on value
    if value <= 25:
        color = '🔴 Extreme Fear'
    elif value <= 45:
        color = '🟠 Fear'
    elif value <= 55:
        color = '🟡 Neutral'
    elif value <= 75:
        color = '🟢 Greed'
    else:
        color = '🟣 Extreme Greed'
    
    print(f"Fear & Greed Index: {value} - {classification}")
    print(f"Sentiment: {color}")
else:
    print("Could not fetch Fear & Greed Index")

## 7. Network Statistics

In [ ]:
# Get Bitcoin network statistics
network_stats = bot.data_fetcher.get_network_stats()

if network_stats:
    print("Bitcoin Network Statistics:")
    print("=" * 40)
    if 'hash_rate' in network_stats:
        print(f"Hash Rate: {network_stats['hash_rate']:,.0f} GH/s")
    if 'difficulty' in network_stats:
        print(f"Difficulty: {network_stats['difficulty']:,.0f}")
    if 'unconfirmed_tx_count' in network_stats:
        print(f"Unconfirmed Transactions: {network_stats['unconfirmed_tx_count']:,}")
    if 'market_cap' in network_stats:
        print(f"Market Cap: ${network_stats['market_cap']:,.0f}")
else:
    print("Could not fetch network statistics")

## 8. Historical Anomaly Analysis

In [ ]:
def analyze_historical_anomalies(df: pd.DataFrame, config: BotConfig) -> pd.DataFrame:
    """Analyze historical data for anomalies."""
    
    results = []
    prices = df['close']
    volumes = df['volume']
    
    # Calculate rolling statistics
    rolling_mean = prices.rolling(window=50).mean()
    rolling_std = prices.rolling(window=50).std()
    
    vol_mean = volumes.rolling(window=50).mean()
    vol_std = volumes.rolling(window=50).std()
    
    for i in range(50, len(df)):
        # Price Z-score
        price_z = (prices.iloc[i] - rolling_mean.iloc[i]) / rolling_std.iloc[i]
        
        # Volume Z-score
        vol_z = (volumes.iloc[i] - vol_mean.iloc[i]) / vol_std.iloc[i]
        
        # Check for anomalies
        anomaly_types = []
        
        if abs(price_z) > config.price_zscore_threshold:
            anomaly_types.append('PRICE')
        if vol_z > config.volume_zscore_threshold:
            anomaly_types.append('VOLUME')
        
        if anomaly_types:
            timestamp = df['datetime'].iloc[i] if 'datetime' in df.columns else df.index[i]
            results.append({
                'timestamp': timestamp,
                'price': prices.iloc[i],
                'volume': volumes.iloc[i],
                'price_zscore': price_z,
                'volume_zscore': vol_z,
                'anomaly_types': ', '.join(anomaly_types)
            })
    
    return pd.DataFrame(results)

# Analyze historical anomalies
historical_anomalies = analyze_historical_anomalies(df.copy(), config)
print(f"Found {len(historical_anomalies)} historical anomaly events:")
historical_anomalies.head(20)

## 9. Multi-Timeframe Oversold / Overbought Analysis

Run a comprehensive analysis across hourly, daily, and weekly timeframes combining RSI, Bollinger Bands, Moving Averages, and the Fear & Greed Index into a single composite score.

In [ ]:
# Run the full multi-timeframe analysis
analysis = bot.run_full_analysis()

### Multi-Timeframe RSI Dashboard

In [ ]:
# Build RSI table across all timeframes
all_rsi = analysis.hourly_rsi + analysis.daily_rsi + analysis.weekly_rsi
if all_rsi:
    rsi_df = pd.DataFrame(all_rsi)
    
    # Color-code RSI values
    def rsi_color(val):
        if val <= 30:
            return 'background-color: #ff4444; color: white'
        elif val <= 40:
            return 'background-color: #ff8888'
        elif val >= 70:
            return 'background-color: #44ff44'
        elif val >= 60:
            return 'background-color: #88ff88'
        return ''
    
    styled = rsi_df.style.applymap(rsi_color, subset=['value'])
    display(styled)
    
    # RSI heatmap
    fig_rsi = go.Figure()
    for tf in ['Hourly', 'Daily', 'Weekly']:
        tf_data = [r for r in all_rsi if r['timeframe'] == tf]
        if tf_data:
            fig_rsi.add_trace(go.Bar(
                name=tf,
                x=[f"RSI-{r['period']}" for r in tf_data],
                y=[r['value'] for r in tf_data],
                text=[r['interpretation'] for r in tf_data],
                textposition='outside',
            ))
    
    fig_rsi.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Overbought")
    fig_rsi.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Oversold")
    fig_rsi.update_layout(
        title="Multi-Timeframe RSI Dashboard",
        yaxis_title="RSI Value",
        barmode='group',
        template='plotly_white',
        height=450,
    )
    fig_rsi.show()
else:
    print("No RSI data available.")

### Moving Average Analysis

In [ ]:
# Price vs Moving Averages
if analysis.ma_analysis:
    ma_df = pd.DataFrame(analysis.ma_analysis)
    display(ma_df)
    
    # Visualize price vs all MAs
    fig_ma = go.Figure()
    
    # Current price line
    fig_ma.add_hline(
        y=analysis.current_price, line_dash="solid", line_color="black",
        annotation_text=f"Current: ${analysis.current_price:,.0f}",
    )
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    for i, ma in enumerate(analysis.ma_analysis):
        color = colors[i % len(colors)]
        above = ma['above']
        fig_ma.add_trace(go.Bar(
            name=f"MA-{ma['period']}",
            x=[f"MA-{ma['period']}"],
            y=[ma['ma_value']],
            marker_color=color,
            text=f"{'ABOVE' if above else 'BELOW'} ({ma['distance_pct']:+.1f}%)",
            textposition='outside',
        ))
    
    fig_ma.update_layout(
        title=f"BTC Price vs Daily Moving Averages (Current: ${analysis.current_price:,.0f})",
        yaxis_title="Price ($)",
        template='plotly_white',
        height=400,
        showlegend=True,
    )
    fig_ma.show()
else:
    print("No moving average data available.")

### Fear & Greed History (30-day trend)

In [ ]:
# Fear & Greed 30-day history chart
if analysis.fng_history:
    fng_dates = [h['timestamp'] for h in analysis.fng_history]
    fng_values = [h['value'] for h in analysis.fng_history]
    fng_labels = [h['classification'] for h in analysis.fng_history]
    
    fig_fng = go.Figure()
    
    # Color zones
    fig_fng.add_hrect(y0=0, y1=25, fillcolor="red", opacity=0.08, line_width=0,
                      annotation_text="Extreme Fear", annotation_position="top left")
    fig_fng.add_hrect(y0=25, y1=45, fillcolor="orange", opacity=0.08, line_width=0,
                      annotation_text="Fear", annotation_position="top left")
    fig_fng.add_hrect(y0=45, y1=55, fillcolor="yellow", opacity=0.08, line_width=0)
    fig_fng.add_hrect(y0=55, y1=75, fillcolor="lightgreen", opacity=0.08, line_width=0,
                      annotation_text="Greed", annotation_position="top left")
    fig_fng.add_hrect(y0=75, y1=100, fillcolor="green", opacity=0.08, line_width=0,
                      annotation_text="Extreme Greed", annotation_position="top left")
    
    fig_fng.add_trace(go.Scatter(
        x=fng_dates, y=fng_values,
        mode='lines+markers',
        name='Fear & Greed Index',
        text=fng_labels,
        line=dict(color='#333', width=2),
        marker=dict(size=6),
    ))
    
    fig_fng.update_layout(
        title=f"Crypto Fear & Greed Index ({len(fng_values)}-day history)",
        yaxis_title="Index Value",
        yaxis=dict(range=[0, 100]),
        template='plotly_white',
        height=400,
    )
    fig_fng.show()
    
    print(f"Average: {np.mean(fng_values):.1f} | "
          f"Min: {min(fng_values)} | Max: {max(fng_values)} | "
          f"Current: {fng_values[0]}")
else:
    print("No Fear & Greed history available.")

### Combined Verdict — Composite Score

In [ ]:
# Composite score visual gauge
if analysis.composite:
    comp = analysis.composite
    score = comp['composite']
    
    # Gauge chart
    fig_gauge = go.Figure(go.Indicator(
        mode="gauge+number+delta",
        value=score,
        title={'text': "Composite Oversold/Overbought Score"},
        delta={'reference': 0, 'increasing': {'color': 'red'}, 'decreasing': {'color': 'green'}},
        gauge={
            'axis': {'range': [-1, 1], 'tickvals': [-1, -0.6, -0.3, 0, 0.3, 0.6, 1]},
            'bar': {'color': 'darkblue'},
            'steps': [
                {'range': [-1, -0.6], 'color': '#006400'},
                {'range': [-0.6, -0.3], 'color': '#228B22'},
                {'range': [-0.3, -0.1], 'color': '#90EE90'},
                {'range': [-0.1, 0.1], 'color': '#FFFFE0'},
                {'range': [0.1, 0.3], 'color': '#FFD700'},
                {'range': [0.3, 0.6], 'color': '#FF8C00'},
                {'range': [0.6, 1], 'color': '#FF0000'},
            ],
            'threshold': {
                'line': {'color': 'black', 'width': 4},
                'thickness': 0.75,
                'value': score,
            },
        },
    ))
    fig_gauge.update_layout(height=350, template='plotly_white')
    fig_gauge.show()
    
    # Component breakdown
    print(f"{'='*60}")
    print(f"COMPOSITE SCORE: {score:+.3f}")
    print(f"{'='*60}")
    print(f"\nComponent Breakdown:")
    for k, v in comp.get('components', {}).items():
        w = comp.get('weights', {}).get(k, 0)
        print(f"  {k.upper():<12} score={v:>+7.3f}  weight={w:.0%}")
    print(f"\n>> VERDICT: {comp['verdict']} <<")
    print(f"{'='*60}")
else:
    print("Composite score not available.")

## 10. Run Continuous Monitoring (Optional)

**Warning:** This will run indefinitely. Use Ctrl+C or restart kernel to stop.

In [ ]:
# Uncomment to run continuous monitoring
# bot.run()

## 11. Export Configuration

In [ ]:
# Export current configuration for command-line usage
env_template = f"""
# Bitcoin Anomaly Bot Configuration
# Add these to your .env file or export them

export BTC_SYMBOL="{config.symbol}"
export BTC_LOOKBACK_DAYS="{config.lookback_days}"
export BTC_UPDATE_INTERVAL="{config.update_interval_seconds}"
export BTC_PRICE_ZSCORE="{config.price_zscore_threshold}"
export BTC_VOLUME_ZSCORE="{config.volume_zscore_threshold}"
export BTC_PRICE_CHANGE_24H_PCT="{config.price_change_threshold_24h_pct}"
export BTC_VOLUME_SPIKE_24H="{config.volume_spike_multiplier_24h}"
export BTC_WHALE_THRESHOLD="{config.whale_threshold_btc}"

# Alert settings
export BTC_CONSOLE_OUTPUT="true"
export ALERT_EMAIL_ENABLED="false"
export ALERT_DISCORD_ENABLED="false"
"""

print(env_template)

## Usage from Command Line

```bash
# Run the bot directly
python strategies/bitcoin_anomaly_bot.py

# Or import and use programmatically
python -c "
from strategies.bitcoin_anomaly_bot import BitcoinAnomalyBot, BotConfig
bot = BitcoinAnomalyBot(BotConfig(update_interval_seconds=60))
bot.run()
"
```